In [1]:
"""This is video reader with all operations inside fnctions, so they can be called on 
    each frame out of a set of frames, and each video out of a set of videos
    """

'This is video reader with all operations inside fnctions, so they can be called on \n    each frame out of a set of frames, and each video out of a set of videos\n    '

In [2]:
"""Imports"""
import matplotlib.pyplot as plt #for maknig plots inside the notebook
import skimage
import skvideo.io
from skvideo.io import vreader, ffprobe
from skimage import measure, morphology #functions for measuring image properties
from skimage.filters import * # functs for thresholding an image (turning greyscale to black and white)
from operator import attrgetter #
import numpy as np
import math
from itertools import chain #links ranges
#from scipy import ndimage 
from pims import pipeline, Video
from glob import glob
import seaborn as sns
from collections import OrderedDict 
#import antigravity

In [3]:
"""Functions"""
# apply total thresholding to each of a list of frames; filter slides with wrong number
def total_threshold_filter(frame, frame_no, transition_threshold, class_thresh,
                           broken_count, class_num, origin, last_class, last_whole, filtrate_len, 
                           sides, classes, transitions): 
    
    m_lab = measure.label
    m_rop = measure.regionprops
    # returns a new list, different from the orginal
    thresh_img = frame > threshold_li(frame) # binary image
    img_labelled = m_lab(thresh_img) # contains connected regions
    properties_list = m_rop(img_labelled, coordinates = 'rc') # data about regions, for each connected region

    #----------getting maximum connected region----------
    biggest_r = max(properties_list, key = attrgetter('area'))
    biggest_r = m_rop(morphology.dilation(morphology.closing(m_lab(biggest_r.filled_image))), coordinates='rc')[0]
    #----------------------filter------------------------- can use actual filter
    test = (biggest_r.filled_area > area_threshold 
                #and biggest_r.eccentricity < ecc_threshold and# must have enough particles
            #and biggest_r.minor_axis_length > minor_thresholds[0] # minor_axis length (< filterd) # particles must be in parallelogram
            #and biggest_r.minor_axis_length < minor_thresholds[1]  # minor_axis length (< filterd) # particles must be in parallelogram
            #and biggest_r.convex_area > convex_thresholds[0]  # convex hull area (> 2500 filtered)  # particles must be in parallelogram
            #and biggest_r.convex_area < convex_thresholds[1]  # convex hull area (> 2500 filtered)  # particles must be in parallelogram
            and biggest_r.major_axis_length > major_thresholds[0]  # major_axis length (< filterd) # particles must be in parallelogram
            and biggest_r.major_axis_length < major_thresholds[1]   # major_axis length (< filterd) # particles must be in parallelogram
               ) # major_axis length (< filterd) # particles must be in parallelogram
    if test: # keep frames that have enough paricles, and are not transitions
        i_threshed = frame > threshold_isodata(frame)
        img2_labelled = m_lab(i_threshed)
        properties2_list = m_rop(img2_labelled, coordinates = 'rc')
        biggest_r2 = max(properties2_list, key = attrgetter('area'))
        biggest_r2 = m_rop(morphology.erosion(morphology.opening(m_lab(biggest_r2.image))), coordinates='rc')[0]
        #-----------------------------classify pt 1---------------------------------------
        coprod = coord_prod(thresh_img, i_threshed)
        side = sideify(biggest_r2)
        clas = classify(side, -coprod)
        sides[side].append(frame_no)
        classes[clas].append(frame_no)
            
        if clas == 'ucf':
            #broken_count += 1
            #class_num = 0
            #print('ucf->reset class')
            pass
        else:
            if origin == 'ucf':
                origin = clas
                #print("reset origin")
            if clas == last_class:
                class_num += 1
                #print("class_num: %s" %class_num)
            else:
                class_num = 0
                #print("class_num: %s" %class_num)
            if class_num >= class_thresh: 
                if broken_count >= transition_threshold or clas != origin:
                    transitions[origin + "->" + clas].append((last_whole, frame_no))
                    #print(origin + "->" + clas)
                    origin = clas
                    #print("origin: %s" %origin)
                broken_count = 0
                #print("broken_count: %s" %broken_count)
                
            
                
            last_whole = frame_no
            filtrate_len += 1 
            last_class = clas
            
            
    else:
        #print("broken")
        class_num = 0
        #print("class_num: %s" %class_num)
        broken_count += 1
        #print("broken_count: %s" %broken_count)

    return [broken_count, class_num, origin, last_class, last_whole, filtrate_len]




        
# take largest isodata image region, and return aclassification
def sideify(idir):
    iner = idir.inertia_tensor[0, 0] + idir.inertia_tensor[1, 1]
    maal = idir.major_axis_length
    mial = idir.minor_axis_length
    sol = idir.solidity 
    ecc = idir.eccentricity
    cva = idir.convex_area
    area = idir.area
    if 1 == 0:
        return "ucf"
    elif ecc > 0.79 and  iner > 330 and iner < 425: #cva > 1920 and cva < 2250:
        return "t"
    elif maal > 66 and maal < 74 and ecc < 0.81: # maal > 67.5 and or (mial < 41 and maal > 65): # it's on top
        return "bm"
    elif  (maal<62.5 and ecc < 0.81): #or mial > 42  #and cva > 2200 and iner < 325: #or (maal < 64 and iner < 360): # it's on bottom
        return "b"
    else:
        return "ucf"
    """iner = idir.inertia_tensor[0, 0] + idir.inertia_tensor[1, 1]
    maal = idir.major_axis_length
    mial = idir.minor_axis_length
    sol = idir.solidity 
    ecc = idir.eccentricity
    cva = idir.convex_area
    if sol <= 0.725 or (iner> 405 and mial > 43.5): # sol <.72
        return "bm"
    elif ecc > 0.8 or maal > 65: # or (mial < 41 and maal > 65): # it's on top #ec >.82
        return "t"
    elif ecc < 0.75 or maal < 64.5: #or mial > 42  #and cva > 2200 and iner < 325: #or (maal < 64 and iner < 360): # it's on bottom
        return "b"
    else:
        return "ucf"
        """

def classify(side, coprod):
    if side == "t":
        if coprod > 0:
            return "tl"
        else:
            return "tr"
    elif side == "b": 
        if coprod > 0:
            return "br"
        else:
            return "bl"
    elif side == "bm":
        return "bm"
    else:
        return "ucf"


def coord_prod(thresh_y, thresh_i):
    m_rop = measure.regionprops
    full = largest_region_extractor(measure.label(thresh_y))
    four = largest_region_extractor(measure.label(thresh_i))
    full_or = m_rop(full.astype(int), coordinates = 'rc')[0].orientation
    one_and_some = full^four
    one = largest_region_extractor(measure.label(one_and_some))


    #Begin finding substitue for major and minor axes herre
    # use min and maax x and  values to find the geometric center
    # or use those to substitue for eigenvectors 
    rotated_one = ndimage.rotate(one, math.degrees(-full_or), reshape = False)
    rotated_full = ndimage.rotate(full, math.degrees(-full_or), reshape = False)
    geo_cent = m_rop(rotated_full.astype(int), coordinates = 'rc')[0].bbox
    #print(geo_cent)
    ave = np.average
    cent_x = ave([geo_cent[1], geo_cent[3]]) 
    cent_y = ave([geo_cent[0], geo_cent[2]])
    #print(cent_x, cent_y)

    raw_coords = (m_rop(rotated_full.astype(int), coordinates = 'rc')[0].coords).T
    ys = raw_coords[0]
    xs = raw_coords[1]
    x = xs - cent_x
    y = ys - cent_y      

    one_com = m_rop(rotated_one.astype(int))[0].centroid

    rel_com_one = (one_com[1] - cent_x, one_com[0] - cent_y)
    product = np.prod(rel_com_one)
    return product
        
def rotate_point(point, angle):
    x0 = point[0]
    y0 = point[1]
    x1 = x0*np.cos(angle) - y0*np.sin(angle)
    y1 = x0*np.sin(angle) + y0*np.cos(angle)
    return (x1, y1)
 
def region_selector(labeled_image, label):
    #print(label)
    x = labeled_image == label
    #plt.imshow(x)
    return x

# make a function that takes a set of labeled regions, and then  returns a boolean array containing only the largest
def largest_region_extractor(labeled_regions_set):
    props_lists = measure.regionprops(labeled_regions_set)
    #print(len(labeled_regions_set), len(props_lists))
    biggest_r_label = max(props_lists, key = attrgetter('area')).label
    return region_selector(labeled_regions_set, biggest_r_label)    
#from pair of labelled, thresholded images, return an image category (of three)

#from pair of labelled,thresholded images, return the x/y sign or x/y magnitude for the transparent dot


In [8]:
"""Pipeline for lists of frames"""
""""""
"""Pipeline for videos:"""
# initialize frame lists
filenames =  glob("./tchakamau/New_setup/*[0-9][0-9].avi")#glob("./tchakamau/New_setup/Multiple*.avi")+ glob("./tchakamau/New_setup/p*shuffles*.avi") +  glob("./tchakamau/New_setup/*unstable*.avi")
print(filenames)
num_top_keys = 4
num_full_keys = 6
num_tran_keys = 5
trans_threshes = [15, 30, 40, 50, 60, 80]#[5, 8, 10, 19, 27, 38, 52, 60]
class_thresh = 22
trans_runs = {}
#trans_fil = 10

ave = np.average

area_threshold = 1950  # must have enough particles
convex_thresholds = (2000, 2800)   # convex hull area (> 2500 filtered)  # particles must be in parallelogram
minor_thresholds = (41 , 50) # minor_axis length (< filterd) # particles must be in parallelogram
major_thresholds = (60, 75)
"""area_threshold = 1800   # must have enough particles
convex_thresholds = (2000, 2800)   # convex hull area (> 2500 filtered)  # particles must be in parallelogram
minor_thresholds = (38, 48) # minor_axis length (< filterd) # particles must be in parallelogram
major_thresholds = (59, 76)"""
thresholds = [area_threshold , major_thresholds ]
params = [1280,  0.7 ,41,0.8,0.91,66.5,74]
print("Number files: ", len(filenames))

for trans_fil in trans_threshes:
    print("T threshold: %s" %trans_fil)
    Top_bottoms = np.zeros((len(filenames), num_top_keys))
    Full_classif = np.zeros((len(filenames), num_full_keys))
    N_transitions = np.zeros((len(filenames), num_tran_keys, num_tran_keys))
    num_total_frames = 0 
    filtrate_len = 0
    for vidnum in range(len(filenames)):
        print("Processing vid %s : %s" %(vidnum, filenames[vidnum]))
        #meta = ffprobe(filenames[vidnum])
        #print(meta)#['@nb_frames'])
        #print(5)
        frames = vreader(filenames[vidnum])
        #frame_vid = frames[:, :, :, 2]# making videos ino a frame list
        #print(frames)



        sides = {"bm":[], "b":[], "t":[], "ucf":[]}
        classes = {"bm":[], "br":[], "bl":[], "tr":[], "tl":[], "ucf":[]}
        transitions = [("bm->bm",[]), ("bm->br",[]), ("bm->bl",[]), ("bm->tr",[]), ("bm->tl",[]),
                      ("br->bm" ,[]), ("br->br",[]), ("br->bl",[]), ("br->tr",[]), ("br->tl",[]),
                      ("bl->bm" ,[]), ("bl->br",[]), ("bl->bl",[]), ("bl->tr",[]), ("bl->tl",[]),
                      ("tr->bm" ,[]), ("tr->br",[]), ("tr->bl",[]), ("tr->tr",[]), ("tr->tl",[]),
                      ("tl->bm" ,[]), ("tl->br",[]), ("tl->bl",[]), ("tl->tr",[]), ("tl->tl",[])]

        transitions = OrderedDict(transitions)
        broken_count = 0
        class_num = 0
        origin = 'ucf'
        last_class = 'ucf'
        last_whole = 0
        for fr in frames:
            frame = fr[:, :, 2]
            num_total_frames += 1
            org = total_threshold_filter(frame, num_total_frames, trans_fil,class_thresh,
                                         broken_count, class_num, origin, last_class, last_whole, filtrate_len, 
                                         sides, classes, transitions)
            broken_count, class_num, origin, last_class, last_whole, filtrate_len = org
        filtered_len = filtrate_len
        #----------------------------------------------------------------
        trans_len = len([y for x in transitions.values() for y in x])
        skeys = list(sides.keys())
        ckeys = list(classes.keys())
        tkeys = list(transitions.keys())
        slcs = [len(sides[x]) for x in skeys]
        clcs = [len(classes[x]) for x in ckeys]
        trans = [len(transitions[x]) for x in tkeys] #num of each transition
        ucf = len(sides["ucf"])


        Top_bottoms[vidnum,:] = [slcs[i] for i in range(len(skeys))]
        Full_classif[vidnum,:] = [clcs[i] for i in range(len(ckeys))]
        N_transitions[vidnum,:] = np.reshape(trans, (num_tran_keys, num_tran_keys))
        print(N_transitions[vidnum,:])
        print("finished vid %s" %vidnum)
    trans_runs[trans_fil] = [[Top_bottoms, Full_classif, N_transitions, trans_threshes, num_top_keys, num_full_keys,
                             num_tran_keys, skeys, ckeys, num_tran_keys, num_total_frames], thresholds, params]
    #---------------------------------------------------------------------------------
    
#-----------------------------------------------------------------------

np.save('trans_rerun1.npy',trans_runs)


['./tchakamau/New_setup\\45621_10Vpp_22.avi', './tchakamau/New_setup\\45621_10Vpp_23.avi', './tchakamau/New_setup\\45681_10Vpp_24.avi', './tchakamau/New_setup\\45681_10Vpp_25.avi', './tchakamau/New_setup\\45681_10Vpp_26.avi', './tchakamau/New_setup\\45681_10Vpp_27.avi', './tchakamau/New_setup\\45701_10Vpp_18.avi', './tchakamau/New_setup\\45701_10Vpp_19.avi', './tchakamau/New_setup\\45701_10Vpp_20.avi', './tchakamau/New_setup\\45701_10Vpp_21.avi']
Number files:  10
T threshold: 15
Processing vid 0 : ./tchakamau/New_setup\45621_10Vpp_22.avi


C:\ProgramData\Anaconda3\lib\site-packages\skimage\filters\thresholding.py:601: RuntimeWarning: divide by zero encountered in log
  (np.log(mean_back) - np.log(mean_fore)))


[[10.  0.  1.  1.  4.]
 [ 0.  0.  0.  0.  0.]
 [ 2.  0.  0.  3.  0.]
 [ 2.  0.  1.  4.  3.]
 [ 2.  0.  2.  2.  2.]]
finished vid 0
Processing vid 1 : ./tchakamau/New_setup\45621_10Vpp_23.avi
[[5. 0. 1. 0. 2.]
 [0. 0. 0. 0. 1.]
 [1. 0. 3. 2. 0.]
 [0. 0. 0. 0. 3.]
 [2. 0. 2. 2. 0.]]
finished vid 1
Processing vid 2 : ./tchakamau/New_setup\45681_10Vpp_24.avi
[[5. 0. 0. 1. 1.]
 [0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [2. 0. 1. 6. 1.]
 [0. 1. 0. 2. 4.]]
finished vid 2
Processing vid 3 : ./tchakamau/New_setup\45681_10Vpp_25.avi
[[0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0.]
 [1. 0. 2. 0. 0.]
 [0. 0. 0. 2. 2.]
 [0. 1. 0. 1. 3.]]
finished vid 3
Processing vid 4 : ./tchakamau/New_setup\45681_10Vpp_26.avi
[[19.  1.  4.  6. 12.]
 [ 1.  3.  5.  2.  3.]
 [ 1.  5.  9.  6.  6.]
 [ 9.  2.  3.  5. 11.]
 [12.  4.  5. 11. 20.]]
finished vid 4
Processing vid 5 : ./tchakamau/New_setup\45681_10Vpp_27.avi
[[ 3.  0.  0.  5.  0.]
 [ 0.  0.  1.  1.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 3.  1.  0. 11.  5.]
 [ 2.  1.  0.  2.  3.]]
fini

[[2. 1. 0. 2. 2.]
 [0. 3. 5. 1. 1.]
 [1. 3. 2. 5. 2.]
 [4. 1. 4. 4. 2.]
 [0. 2. 2. 3. 0.]]
finished vid 8
Processing vid 9 : ./tchakamau/New_setup\45701_10Vpp_21.avi
[[4. 1. 1. 1. 3.]
 [1. 1. 1. 0. 0.]
 [1. 1. 1. 0. 0.]
 [1. 0. 0. 1. 2.]
 [3. 0. 0. 1. 6.]]
finished vid 9
T threshold: 80
Processing vid 0 : ./tchakamau/New_setup\45621_10Vpp_22.avi
[[2. 0. 1. 1. 4.]
 [0. 0. 0. 0. 0.]
 [2. 0. 0. 3. 0.]
 [2. 0. 1. 0. 3.]
 [2. 0. 2. 2. 1.]]
finished vid 0
Processing vid 1 : ./tchakamau/New_setup\45621_10Vpp_23.avi
[[1. 0. 1. 0. 2.]
 [0. 0. 0. 0. 1.]
 [1. 0. 2. 2. 0.]
 [0. 0. 0. 0. 3.]
 [2. 0. 2. 2. 0.]]
finished vid 1
Processing vid 2 : ./tchakamau/New_setup\45681_10Vpp_24.avi
[[1. 0. 0. 1. 1.]
 [0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [2. 0. 1. 1. 1.]
 [0. 1. 0. 2. 1.]]
finished vid 2
Processing vid 3 : ./tchakamau/New_setup\45681_10Vpp_25.avi
[[0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0.]
 [0. 0. 0. 1. 2.]
 [0. 1. 0. 1. 1.]]
finished vid 3
Processing vid 4 : ./tchakamau/New_setup\45681_1

In [10]:
trans_runs= np.load('trans_rerun1.npy')[()]
"""num_top_keys = 4
num_full_keys = 6
num_tran_keys = 5
trans_threshes = [5, 8, 10, 19, 27, 38, 52, 60]
sides = {"bm":[], "b":[], "t":[], "ucf":[]}
classes = {"bm":[], "br":[], "bl":[], "tr":[], "tl":[], "ucf":[]}
transitions = {"bm->bm" :[], "bm->br":[], "bm->bl":[], "bm->tr":[], "bm->tl":[],
              "br->bm" :[], "br->br":[], "br->bl":[], "br->tr":[], "br->tl":[],
              "bl->bm" :[], "bl->br":[], "bl->bl":[], "bl->tr":[], "bl->tl":[],
              "tr->bm" :[], "tr->br":[], "tr->bl":[], "tr->tr":[], "tr->tl":[],
              "tl->bm" :[], "tl->br":[], "tl->bl":[], "tl->tr":[], "tl->tl":[]}"""
trans_threshes = [15, 30, 40, 50, 60, 80]
sides = {"bm":[], "b":[], "t":[], "ucf":[]}
classes = {"bm":[], "br":[], "bl":[], "tr":[], "tl":[], "ucf":[]}
skeys = list(sides.keys())
ckeys = list(classes.keys())
for trans_fil in trans_threshes:
    print("\nTRANSITION THRESHOLD ", trans_fil)
    
    #Top_bottoms, Full_classif, N_transitions = trans_runs[trans_fil]
    Top_bottoms, Full_classif, N_transitions, trans_threshes, num_top_keys, num_full_keys, num_tran_keys, skeys, ckeys, num_tran_keys, num_total_frames = trans_runs[trans_fil][0]
    sums = np.sum
    Tb_sums = [sums(Top_bottoms[:, c]) for c in range(num_top_keys)]
    Fc_sums = [sums(Full_classif[:, c]) for c in range(num_full_keys)]
    Nt_sums = sums(N_transitions, axis = 0)#[sums(N_transitions[:, c]) for c in range(num_tran_keys)]
    #print("\nNumber of frames obbserved: ", num_total_frames)

    Tb_percs = [Tb_sums[c] *100 /sums(Tb_sums) for c in range(num_top_keys)]
    Fc_percs = [Fc_sums[c] *100/sums(Fc_sums) for c in range(num_full_keys)]
    Nt_percs = Nt_sums *100/sums(Nt_sums)
    #print(sums(N_transitions), sums(Nt_sums))


    print("\nsums in broad classifications")
    [print(skeys[i], "sum %.f" %Tb_sums[i]) for i in range(len(skeys))]
    print("Total: ", sums(Tb_sums) )
    print("\nsums in narrow classifications")
    [print(ckeys[i], "sum %.f" %Fc_sums[i]) for i in range(len(ckeys))]
    print("Total: ", sums(Fc_sums ) )
    print("\nsums of transitions broad")
    T_eb = np.vstack([Nt_sums[0], 
                                 (Nt_sums[1] + Nt_sums[2]) , 
                                 (Nt_sums[3] + Nt_sums[4])])
    Nt_sums_broad = np.column_stack([T_eb[:, 0], 
                                 (T_eb[:, 1]+ T_eb[:, 2]), 
                                 (T_eb[:, 3]+ T_eb[:, 4])])
    print(Nt_sums_broad)
    print("\nsums of transitions narrow")
    print(Nt_sums)
    print("Total: ", sums(Nt_sums) )



    """print("\n%s in broad classifications")
    [print(skeys[i], " %.f%%" %Tb_percs[i]) for i in range(len(skeys))]
    print("\n%s in narrow classifications")
    [print(ckeys[i], " %.f%%" %Fc_percs[i]) for i in range(len(ckeys))]"""
    #ERROR ANALSIS HERE
    print("\nErrors in transition matrix ((row_sums - column_sums) / 2)")
    print(((sums(Nt_sums, axis = 1) - sums(Nt_sums, axis =0))/2))
    print("\nError %%s in transition matrix ((row_sums - column_sums) *100 / (row_sums + column_sums))")
    print(((sums(Nt_sums, axis = 1) - sums(Nt_sums, axis =0))*100 /(sums(Nt_sums, axis = 1) + sums(Nt_sums, axis =0))))
    
    print("\nTransition matrix, broad")
    T_eb = np.vstack([Nt_percs[0], 
                                 (Nt_percs[1] + Nt_percs[2]) , 
                                 (Nt_percs[3] + Nt_percs[4])])
    Nt_percs_broad = np.column_stack([T_eb[:, 0], 
                                 (T_eb[:, 1]+ T_eb[:, 2]), 
                                 (T_eb[:, 3]+ T_eb[:, 4])])
    print(Nt_percs_broad)
    """print("row_sums of transitin matrix, broad")
    print(sums(Nt_percs_broad, axis = 1))
    print("\n%s of transitions broad(->bm, ->b, ->t):")
    #print("->bm %.f%%" %(Nt_percs[0, 0] + sums(Nt_percs[1:3, 0]) + sums(Nt_percs[3:5, 0])))
    print([sums(Nt_sums, axis = 0)[0]*100/sums(Nt_sums), 
           (sums(Nt_sums, axis = 0)[1] + sums(Nt_sums, axis = 0)[2])*100/(sums(Nt_sums)),
           (sums(Nt_sums, axis = 0)[3] + sums(Nt_sums, axis = 0)[4])*100/(sums(Nt_sums))])
    print("\n%s of transitions broad(bm->, b->, t->):")
    #print("->bm %.f%%" %(Nt_percs[0, 0] + sums(Nt_percs[1:3, 0]) + sums(Nt_percs[3:5, 0])))
    print([sums(Nt_sums, axis = 1)[0]*100/sums(Nt_sums),
           (sums(Nt_sums, axis = 1)[1] + sums(Nt_sums, axis = 1)[2])*100/(sums(Nt_sums)),
           (sums(Nt_sums, axis = 1)[3] + sums(Nt_sums, axis =1)[4])*100/(sums(Nt_sums))])#(Nt_percs[i,1] + Nt_percs[i,1+1] + Nt_percs[i+1, 1] + Nt_percs[i+1, 1+1] + sums(Nt_percs[0, 1:3]))) for i in [1,3]]
    """
    #print("->t %.f%%" %(Nt_percs[i,2] + Nt_percs[i,2+1] + Nt_percs[i+1, 2] + Nt_percs[i+1, 2+1] + sums(Nt_percs[0, 3:5]))) for i in [1,3]]
    print("\nTransition matrix, narrow")
    print(Nt_percs)
    print(sums(Nt_percs))
    print("\n%s of transitions to states narrow:")
    [print(sums(Nt_percs, axis = 0)*100/sums(Nt_percs))]
    print("\n%s of transitions from states narrow:")
    [print(sums(Nt_percs, axis = 1)*100/sums(Nt_percs))]
    print("\n%s of transitions to states broad:")
    [print(sums(Nt_percs_broad, axis = 0)*100/sums(Nt_percs_broad))]
    print("\n%s of transitions from states broad:")
    [print(sums(Nt_percs_broad, axis = 1)*100/sums(Nt_percs_broad))]
    #print("\n%s of transitions narrow")


TRANSITION THRESHOLD  15

sums in broad classifications
bm sum 72473
b sum 110651
t sum 169447
ucf sum 19057
Total:  371628.0

sums in narrow classifications
bm sum 72473
br sum 72405
bl sum 38246
tr sum 86741
tl sum 82706
ucf sum 19057
Total:  371628.0

sums of transitions broad
[[ 59.  10.  46.]
 [  9.  74.  51.]
 [ 47.  46. 192.]]

sums of transitions narrow
[[59.  3.  7. 19. 27.]
 [ 2. 19. 16.  7. 10.]
 [ 7. 14. 25. 23. 11.]
 [24.  7. 15. 63. 34.]
 [23. 10. 14. 32. 63.]]
Total:  534.0

Errors in transition matrix ((row_sums - column_sums) / 2)
[ 0.   0.5  1.5 -0.5 -1.5]

Error %%s in transition matrix ((row_sums - column_sums) *100 / (row_sums + column_sums))
[ 0.          0.93457944  1.91082803 -0.34843206 -1.04529617]

Transition matrix, broad
[[11.04868914  1.87265918  8.61423221]
 [ 1.68539326 13.8576779   9.5505618 ]
 [ 8.80149813  8.61423221 35.95505618]]

Transition matrix, narrow
[[11.04868914  0.56179775  1.31086142  3.55805243  5.05617978]
 [ 0.37453184  3.55805243  2.99

In [5]:
trans_runs= np.load('trans_runs2.npy')[()]
"""num_top_keys = 4
num_full_keys = 6
num_tran_keys = 5
trans_threshes = [5, 8, 10, 19, 27, 38, 52, 60]
sides = {"bm":[], "b":[], "t":[], "ucf":[]}
classes = {"bm":[], "br":[], "bl":[], "tr":[], "tl":[], "ucf":[]}
transitions = {"bm->bm" :[], "bm->br":[], "bm->bl":[], "bm->tr":[], "bm->tl":[],
              "br->bm" :[], "br->br":[], "br->bl":[], "br->tr":[], "br->tl":[],
              "bl->bm" :[], "bl->br":[], "bl->bl":[], "bl->tr":[], "bl->tl":[],
              "tr->bm" :[], "tr->br":[], "tr->bl":[], "tr->tr":[], "tr->tl":[],
              "tl->bm" :[], "tl->br":[], "tl->bl":[], "tl->tr":[], "tl->tl":[]}"""
trans_threshes = [5, 8, 10, 19, 27, 38, 52, 60]
sides = {"bm":[], "b":[], "t":[], "ucf":[]}
classes = {"bm":[], "br":[], "bl":[], "tr":[], "tl":[], "ucf":[]}
skeys = list(sides.keys())
ckeys = list(classes.keys())
for trans_fil in trans_threshes:
    print("\nTRANSITION THRESHOLD ", trans_fil)
    
    #Top_bottoms, Full_classif, N_transitions = trans_runs[trans_fil]
    Top_bottoms, Full_classif, N_transitions, trans_threshes, num_top_keys, num_full_keys, num_tran_keys, skeys, ckeys, num_tran_keys, num_total_frames = trans_runs[trans_fil]
    sums = np.sum
    Tb_sums = [sums(Top_bottoms[:, c]) for c in range(num_top_keys)]
    Fc_sums = [sums(Full_classif[:, c]) for c in range(num_full_keys)]
    Nt_sums = sums(N_transitions, axis = 0)#[sums(N_transitions[:, c]) for c in range(num_tran_keys)]
    #print("\nNumber of frames obbserved: ", num_total_frames)

    Tb_percs = [Tb_sums[c] *100 /sums(Tb_sums) for c in range(num_top_keys)]
    Fc_percs = [Fc_sums[c] *100/sums(Fc_sums) for c in range(num_full_keys)]
    Nt_percs = Nt_sums *100/sums(Nt_sums)
    #print(sums(N_transitions), sums(Nt_sums))


    print("\nsums in broad classifications")
    [print(skeys[i], "sum %.f" %Tb_sums[i]) for i in range(len(skeys))]
    print("Total: ", sums(Tb_sums) )
    print("\nsums in narrow classifications")
    [print(ckeys[i], "sum %.f" %Fc_sums[i]) for i in range(len(ckeys))]
    print("Total: ", sums(Fc_sums ) )
    print("\nsums of transitions broad")
    T_eb = np.vstack([Nt_sums[0], 
                                 (Nt_sums[1] + Nt_sums[2]) , 
                                 (Nt_sums[3] + Nt_sums[4])])
    Nt_sums_broad = np.column_stack([T_eb[:, 0], 
                                 (T_eb[:, 1]+ T_eb[:, 2]), 
                                 (T_eb[:, 3]+ T_eb[:, 4])])
    print(Nt_sums_broad)
    print("\nsums of transitions narrow")
    print(Nt_sums)
    print("Total: ", sums(Nt_sums) )



    """print("\n%s in broad classifications")
    [print(skeys[i], " %.f%%" %Tb_percs[i]) for i in range(len(skeys))]
    print("\n%s in narrow classifications")
    [print(ckeys[i], " %.f%%" %Fc_percs[i]) for i in range(len(ckeys))]"""
    #ERROR ANALSIS HERE
    print("\nErrors in transition matrix ((row_sums - column_sums) / 2)")
    print(((sums(Nt_sums, axis = 1) - sums(Nt_sums, axis =0))/2))
    print("\nError %%s in transition matrix ((row_sums - column_sums) *100 / (row_sums + column_sums))")
    print(((sums(Nt_sums, axis = 1) - sums(Nt_sums, axis =0))*100 /(sums(Nt_sums, axis = 1) + sums(Nt_sums, axis =0))))
    
    print("\nTransition matrix, broad")
    T_eb = np.vstack([Nt_percs[0], 
                                 (Nt_percs[1] + Nt_percs[2]) , 
                                 (Nt_percs[3] + Nt_percs[4])])
    Nt_percs_broad = np.column_stack([T_eb[:, 0], 
                                 (T_eb[:, 1]+ T_eb[:, 2]), 
                                 (T_eb[:, 3]+ T_eb[:, 4])])
    print(Nt_percs_broad)
    """print("row_sums of transitin matrix, broad")
    print(sums(Nt_percs_broad, axis = 1))
    print("\n%s of transitions broad(->bm, ->b, ->t):")
    #print("->bm %.f%%" %(Nt_percs[0, 0] + sums(Nt_percs[1:3, 0]) + sums(Nt_percs[3:5, 0])))
    print([sums(Nt_sums, axis = 0)[0]*100/sums(Nt_sums), 
           (sums(Nt_sums, axis = 0)[1] + sums(Nt_sums, axis = 0)[2])*100/(sums(Nt_sums)),
           (sums(Nt_sums, axis = 0)[3] + sums(Nt_sums, axis = 0)[4])*100/(sums(Nt_sums))])
    print("\n%s of transitions broad(bm->, b->, t->):")
    #print("->bm %.f%%" %(Nt_percs[0, 0] + sums(Nt_percs[1:3, 0]) + sums(Nt_percs[3:5, 0])))
    print([sums(Nt_sums, axis = 1)[0]*100/sums(Nt_sums),
           (sums(Nt_sums, axis = 1)[1] + sums(Nt_sums, axis = 1)[2])*100/(sums(Nt_sums)),
           (sums(Nt_sums, axis = 1)[3] + sums(Nt_sums, axis =1)[4])*100/(sums(Nt_sums))])#(Nt_percs[i,1] + Nt_percs[i,1+1] + Nt_percs[i+1, 1] + Nt_percs[i+1, 1+1] + sums(Nt_percs[0, 1:3]))) for i in [1,3]]
    """
    #print("->t %.f%%" %(Nt_percs[i,2] + Nt_percs[i,2+1] + Nt_percs[i+1, 2] + Nt_percs[i+1, 2+1] + sums(Nt_percs[0, 3:5]))) for i in [1,3]]
    print("\nTransition matrix, narrow")
    print(Nt_percs)
    print(sums(Nt_percs))
    print("\n%s of transitions to states narrow:")
    [print(sums(Nt_percs, axis = 0)*100/sums(Nt_percs))]
    print("\n%s of transitions from states narrow:")
    [print(sums(Nt_percs, axis = 1)*100/sums(Nt_percs))]
    print("\n%s of transitions to states broad:")
    [print(sums(Nt_percs_broad, axis = 0)*100/sums(Nt_percs_broad))]
    print("\n%s of transitions from states broad:")
    [print(sums(Nt_percs_broad, axis = 1)*100/sums(Nt_percs_broad))]
    #print("\n%s of transitions narrow")


TRANSITION THRESHOLD  5

sums in broad classifications
bm sum 84370
b sum 109454
t sum 83204
ucf sum 92566
Total:  369594.0

sums in narrow classifications
bm sum 84370
br sum 73791
bl sum 35663
tr sum 40931
tl sum 42273
ucf sum 92566
Total:  369594.0

sums of transitions broad
[[ 763.   46.  326.]
 [  49.  604.   78.]
 [ 324.   78. 1533.]]

sums of transitions narrow
[[763.  28.  18.  60. 266.]
 [ 22. 237.  32.   9.  30.]
 [ 27.  25. 310.  20.  19.]
 [ 60.   8.  22. 783.  25.]
 [264.  32.  16.  26. 699.]]
Total:  3801.0

Errors in transition matrix ((row_sums - column_sums) / 2)
[-0.5  0.   1.5  0.  -1. ]

Error %%s in transition matrix ((row_sums - column_sums) *100 / (row_sums + column_sums))
[-0.04403347  0.          0.37546934  0.         -0.09633911]

Transition matrix, broad
[[20.07366483  1.21020784  8.57669034]
 [ 1.28913444 15.89055512  2.05209155]
 [ 8.52407261  2.05209155 40.33149171]]

Transition matrix, narrow
[[20.07366483  0.73664825  0.47355959  1.57853197  6.99815838

In [6]:
trans_runs= np.load('trans_runs1.npy')[()]
"""num_top_keys = 4
num_full_keys = 6
num_tran_keys = 5
trans_threshes = [5, 8, 10, 19, 27, 38, 52, 60]
sides = {"bm":[], "b":[], "t":[], "ucf":[]}
classes = {"bm":[], "br":[], "bl":[], "tr":[], "tl":[], "ucf":[]}
transitions = {"bm->bm" :[], "bm->br":[], "bm->bl":[], "bm->tr":[], "bm->tl":[],
              "br->bm" :[], "br->br":[], "br->bl":[], "br->tr":[], "br->tl":[],
              "bl->bm" :[], "bl->br":[], "bl->bl":[], "bl->tr":[], "bl->tl":[],
              "tr->bm" :[], "tr->br":[], "tr->bl":[], "tr->tr":[], "tr->tl":[],
              "tl->bm" :[], "tl->br":[], "tl->bl":[], "tl->tr":[], "tl->tl":[]}"""

skeys = list(sides.keys())
ckeys = list(classes.keys())
for trans_fil in trans_threshes:
    print("\nTRANSITION THRESHOLD ", trans_fil)
    
    #Top_bottoms, Full_classif, N_transitions = trans_runs[trans_fil]
    Top_bottoms, Full_classif, N_transitions, trans_threshes, num_top_keys, num_full_keys, num_tran_keys, skeys, ckeys, num_tran_keys, num_total_frames = trans_runs[trans_fil]
    sums = np.sum
    Tb_sums = [sums(Top_bottoms[:, c]) for c in range(num_top_keys)]
    Fc_sums = [sums(Full_classif[:, c]) for c in range(num_full_keys)]
    Nt_sums = sums(N_transitions, axis = 0)#[sums(N_transitions[:, c]) for c in range(num_tran_keys)]
    #print("\nNumber of frames obbserved: ", num_total_frames)

    Tb_percs = [Tb_sums[c] *100 /sums(Tb_sums) for c in range(num_top_keys)]
    Fc_percs = [Fc_sums[c] *100/sums(Fc_sums) for c in range(num_full_keys)]
    Nt_percs = Nt_sums *100/sums(Nt_sums)
    #print(sums(N_transitions), sums(Nt_sums))


    print("\nsums in broad classifications")
    [print(skeys[i], "sum %.f" %Tb_sums[i]) for i in range(len(skeys))]
    print("Total: ", sums(Tb_sums) )
    print("\nsums in narrow classifications")
    [print(ckeys[i], "sum %.f" %Fc_sums[i]) for i in range(len(ckeys))]
    print("Total: ", sums(Fc_sums ) )
    print("\nsums of transitions broad")
    T_eb = np.vstack([Nt_sums[0], 
                                 (Nt_sums[1] + Nt_sums[2]) , 
                                 (Nt_sums[3] + Nt_sums[4])])
    Nt_sums_broad = np.column_stack([T_eb[:, 0], 
                                 (T_eb[:, 1]+ T_eb[:, 2]), 
                                 (T_eb[:, 3]+ T_eb[:, 4])])
    print(Nt_sums_broad)
    print("\nsums of transitions narrow")
    print(Nt_sums)
    print("Total: ", sums(Nt_sums) )



    """print("\n%s in broad classifications")
    [print(skeys[i], " %.f%%" %Tb_percs[i]) for i in range(len(skeys))]
    print("\n%s in narrow classifications")
    [print(ckeys[i], " %.f%%" %Fc_percs[i]) for i in range(len(ckeys))]"""
    #ERROR ANALSIS HERE
    print("Errors in transition matrix (row_sums - column_sums / 2)")
    print((sums(Nt_percs_broad, axis = 1) - sums(Nt_percs_broad, axis = 1) / 2))
    
    print("Transition matrix, broad")
    T_eb = np.vstack([Nt_percs[0], 
                                 (Nt_percs[1] + Nt_percs[2]) , 
                                 (Nt_percs[3] + Nt_percs[4])])
    Nt_percs_broad = np.column_stack([T_eb[:, 0], 
                                 (T_eb[:, 1]+ T_eb[:, 2]), 
                                 (T_eb[:, 3]+ T_eb[:, 4])])
    print(Nt_percs_broad)
    print("row_sums of transitin matrix, broad")
    print(sums(Nt_percs_broad, axis = 1))
    print("\n%s of transitions broad(->bm, ->b, ->t):")
    #print("->bm %.f%%" %(Nt_percs[0, 0] + sums(Nt_percs[1:3, 0]) + sums(Nt_percs[3:5, 0])))
    print([sums(Nt_sums, axis = 0)[0]*100/sums(Nt_sums), 
           (sums(Nt_sums, axis = 0)[1] + sums(Nt_sums, axis = 0)[2])*100/(sums(Nt_sums)),
           (sums(Nt_sums, axis = 0)[3] + sums(Nt_sums, axis = 0)[4])*100/(sums(Nt_sums))])
    print("\n%s of transitions broad(bm->, b->, t->):")
    #print("->bm %.f%%" %(Nt_percs[0, 0] + sums(Nt_percs[1:3, 0]) + sums(Nt_percs[3:5, 0])))
    print([sums(Nt_sums, axis = 1)[0]*100/sums(Nt_sums),
           (sums(Nt_sums, axis = 1)[1] + sums(Nt_sums, axis = 1)[2])*100/(sums(Nt_sums)),
           (sums(Nt_sums, axis = 1)[3] + sums(Nt_sums, axis =1)[4])*100/(sums(Nt_sums))])#(Nt_percs[i,1] + Nt_percs[i,1+1] + Nt_percs[i+1, 1] + Nt_percs[i+1, 1+1] + sums(Nt_percs[0, 1:3]))) for i in [1,3]]
    #print("->t %.f%%" %(Nt_percs[i,2] + Nt_percs[i,2+1] + Nt_percs[i+1, 2] + Nt_percs[i+1, 2+1] + sums(Nt_percs[0, 3:5]))) for i in [1,3]]
    print("\nTransition matrix, narrow")
    print(Nt_percs)
    print(sums(Nt_percs))
    print("\n%s of transitions to states narrow:")
    [print(sums(Nt_percs, axis = 0)*100/sums(Nt_percs))]
    print("\n%s of transitions from states narrow:")
    [print(sums(Nt_percs, axis = 1)*100/sums(Nt_percs))]
    #print("\n%s of transitions narrow")
    #[print(tkeys[i], " %.f%%" %Nt_percs.flatten()[i]) for i in range(len(tkeys))]


TRANSITION THRESHOLD  5

sums in broad classifications
bm sum 81027
b sum 96792
t sum 75167
ucf sum 82488
Total:  335474.0

sums in narrow classifications
bm sum 81027
br sum 67532
bl sum 29260
tr sum 36261
tl sum 38906
ucf sum 82488
Total:  335474.0

sums of transitions broad
[[477.  94. 109.]
 [ 34. 826.  43.]
 [ 73.  47. 935.]]

sums of transitions narrow
[[477.  54.  40.  52.  57.]
 [ 18. 284.  35.   9.  15.]
 [ 16.  30. 477.  17.   2.]
 [ 29.   4.  21. 407.  35.]
 [ 44.  12.  10.  35. 458.]]
Total:  2638.0
Errors in transition matrix (row_sums - column_sums / 2)
[11.24031008  9.84496124 28.91472868]
Transition matrix, broad
[[18.08188021  3.56330553  4.13191812]
 [ 1.28885519 31.3115997   1.63002274]
 [ 2.76724792  1.78165277 35.44351782]]
row_sums of transitin matrix, broad
[25.77710387 34.23047763 39.9924185 ]

%s of transitions broad(->bm, ->b, ->t):
[22.137983320697497, 36.6565579984837, 41.2054586808188]

%s of transitions broad(bm->, b->, t->):
[25.77710386656558, 34.230477

In [7]:
trans_runs= np.load('trans_runs.npy')[()]
num_top_keys = 4
num_full_keys = 6
num_tran_keys = 5
trans_threshes = [5, 8, 10, 19, 27, 38, 52, 60]
sides = {"bm":[], "b":[], "t":[], "ucf":[]}
classes = {"bm":[], "br":[], "bl":[], "tr":[], "tl":[], "ucf":[]}
transitions = {"bm->bm" :[], "bm->br":[], "bm->bl":[], "bm->tr":[], "bm->tl":[],
              "br->bm" :[], "br->br":[], "br->bl":[], "br->tr":[], "br->tl":[],
              "bl->bm" :[], "bl->br":[], "bl->bl":[], "bl->tr":[], "bl->tl":[],
              "tr->bm" :[], "tr->br":[], "tr->bl":[], "tr->tr":[], "tr->tl":[],
              "tl->bm" :[], "tl->br":[], "tl->bl":[], "tl->tr":[], "tl->tl":[]}

skeys = list(sides.keys())
ckeys = list(classes.keys())
for trans_fil in trans_threshes:
    print("\nTRANSITION THRESHOLD ", trans_fil)
    
    Top_bottoms, Full_classif, N_transitions = trans_runs[trans_fil]
    #Top_bottoms, Full_classif, N_transitions, trans_threshes, num_top_keys, num_full_keys,
                             #num_tran_keys, skeys, ckeys, num_tran_keys, num_total_frames = trans_runs[trans_fil]
    sums = np.sum
    Tb_sums = [sums(Top_bottoms[:, c]) for c in range(num_top_keys)]
    Fc_sums = [sums(Full_classif[:, c]) for c in range(num_full_keys)]
    Nt_sums = sums(N_transitions, axis = 0)#[sums(N_transitions[:, c]) for c in range(num_tran_keys)]
    #print("\nNumber of frames obbserved: ", num_total_frames)

    Tb_percs = [Tb_sums[c] *100 /sums(Tb_sums) for c in range(num_top_keys)]
    Fc_percs = [Fc_sums[c] *100/sums(Fc_sums) for c in range(num_full_keys)]
    Nt_percs = Nt_sums *100/sums(Nt_sums)
    #print(sums(N_transitions), sums(Nt_sums))


    print("\nsums in broad classifications")
    [print(skeys[i], "sum %.f" %Tb_sums[i]) for i in range(len(skeys))]
    print("Total: ", sums(Tb_sums) )
    print("\nsums in narrow classifications")
    [print(ckeys[i], "sum %.f" %Fc_sums[i]) for i in range(len(ckeys))]
    print("Total: ", sums(Fc_sums ) )
    print("\nsums of transitions broad")
    T_eb = np.vstack([Nt_sums[0], 
                                 (Nt_sums[1] + Nt_sums[2]) , 
                                 (Nt_sums[3] + Nt_sums[4])])
    Nt_sums_broad = np.column_stack([T_eb[:, 0], 
                                 (T_eb[:, 1]+ T_eb[:, 2]), 
                                 (T_eb[:, 3]+ T_eb[:, 4])])
    print(Nt_sums_broad)
    print("\nsums of transitions narrow")
    print(Nt_sums)
    print("Total: ", sums(Nt_sums) )



    print("\n%s in broad classifications")
    [print(skeys[i], " %.f%%" %Tb_percs[i]) for i in range(len(skeys))]
    print("\n%s in narrow classifications")
    [print(ckeys[i], " %.f%%" %Fc_percs[i]) for i in range(len(ckeys))]
    print("Transition matrix, broad")
    T_eb = np.vstack([Nt_percs[0], 
                                 (Nt_percs[1] + Nt_percs[2]) , 
                                 (Nt_percs[3] + Nt_percs[4])])
    Nt_percs_broad = np.column_stack([T_eb[:, 0], 
                                 (T_eb[:, 1]+ T_eb[:, 2]), 
                                 (T_eb[:, 3]+ T_eb[:, 4])])
    print(Nt_percs_broad)
    print("row_sums of transitin matrix, broad")
    print(sums(Nt_percs_broad, axis = 1))
    print("\n%s of transitions broad(->bm, ->b, ->t):")
    #print("->bm %.f%%" %(Nt_percs[0, 0] + sums(Nt_percs[1:3, 0]) + sums(Nt_percs[3:5, 0])))
    print([sums(Nt_sums, axis = 0)[0]*100/sums(Nt_sums), 
           (sums(Nt_sums, axis = 0)[1] + sums(Nt_sums, axis = 0)[2])*100/(sums(Nt_sums)),
           (sums(Nt_sums, axis = 0)[3] + sums(Nt_sums, axis = 0)[4])*100/(sums(Nt_sums))])
    print("\n%s of transitions broad(bm->, b->, t->):")
    #print("->bm %.f%%" %(Nt_percs[0, 0] + sums(Nt_percs[1:3, 0]) + sums(Nt_percs[3:5, 0])))
    print([sums(Nt_sums, axis = 1)[0]*100/sums(Nt_sums),
           (sums(Nt_sums, axis = 1)[1] + sums(Nt_sums, axis = 1)[2])*100/(sums(Nt_sums)),
           (sums(Nt_sums, axis = 1)[3] + sums(Nt_sums, axis =1)[4])*100/(sums(Nt_sums))])#(Nt_percs[i,1] + Nt_percs[i,1+1] + Nt_percs[i+1, 1] + Nt_percs[i+1, 1+1] + sums(Nt_percs[0, 1:3]))) for i in [1,3]]
    #print("->t %.f%%" %(Nt_percs[i,2] + Nt_percs[i,2+1] + Nt_percs[i+1, 2] + Nt_percs[i+1, 2+1] + sums(Nt_percs[0, 3:5]))) for i in [1,3]]
    print("\nTransition matrix, narrow")
    print(Nt_percs)
    print(sums(Nt_percs))
    print("\n%s of transitions to states narrow:")
    [print(sums(Nt_percs, axis = 0)*100/sums(Nt_percs))]
    print("\n%s of transitions from states narrow:")
    [print(sums(Nt_percs, axis = 1)*100/sums(Nt_percs))]
    #print("\n%s of transitions narrow")
    #[print(tkeys[i], " %.f%%" %Nt_percs.flatten()[i]) for i in range(len(tkeys))]


TRANSITION THRESHOLD  5

sums in broad classifications
bm sum 70883
b sum 120079
t sum 176777
ucf sum 1855
Total:  369594.0

sums in narrow classifications
bm sum 70883
br sum 78820
bl sum 41259
tr sum 86994
tl sum 89783
ucf sum 1855
Total:  369594.0

sums of transitions broad
[[174.  41.  81.]
 [ 88. 588. 171.]
 [114. 180. 352.]]

sums of transitions narrow
[[174.  23.  18.  40.  41.]
 [ 52. 161.  89.  47.  42.]
 [ 36.  84. 254.  43.  39.]
 [ 62.  28.  60. 102.  56.]
 [ 52.  51.  41.  54. 140.]]
Total:  1789.0

%s in broad classifications
bm  19%
b  32%
t  48%
ucf  1%

%s in narrow classifications
bm  19%
br  21%
bl  11%
tr  24%
tl  24%
ucf  1%
Transition matrix, broad
[[ 9.72610397  2.29178312  4.52766909]
 [ 4.91894913 32.86752376  9.55841252]
 [ 6.37227501 10.06148686 19.67579653]]
row_sums of transitin matrix, broad
[16.54555618 47.34488541 36.10955841]

%s of transitions broad(->bm, ->b, ->t):
[21.01732811626607, 45.22079373951929, 33.76187814421465]

%s of transitions broad(bm-

bl  11%
tr  24%
tl  24%
ucf  1%
Transition matrix, broad
[[ 3.95480226  3.20150659  6.21468927]
 [ 6.77966102 26.55367232 17.13747646]
 [ 6.02636535 13.37099812 16.76082863]]
row_sums of transitin matrix, broad
[13.37099812 50.47080979 36.15819209]

%s of transitions broad(->bm, ->b, ->t):
[16.760828625235405, 43.12617702448211, 40.112994350282484]

%s of transitions broad(bm->, b->, t->):
[13.370998116760829, 50.47080979284369, 36.15819209039548]

Transition matrix, narrow
[[3.95480226 1.31826742 1.88323917 3.01318267 3.20150659]
 [4.33145009 5.64971751 7.53295669 5.83804143 3.38983051]
 [2.44821092 7.53295669 5.83804143 3.95480226 3.95480226]
 [3.38983051 2.44821092 4.33145009 4.33145009 4.70809793]
 [2.63653484 3.38983051 3.20150659 3.01318267 4.70809793]]
99.99999999999999

%s of transitions to states narrow:
[16.76082863 20.33898305 22.78719397 20.15065913 19.96233522]

%s of transitions from states narrow:
[13.37099812 26.74199623 23.72881356 19.20903955 16.94915254]

TRANSITION 